In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

In [2]:
# 1 Indexing 
            # a. document ingestion
            # b. Text Chunking
            # c. Embedding Generation
            # d. Storage in a Vector Store
# 2 Retrieval 
# 3 Augmentation 
# 4 Generation

## Step 1a - Indexing (Document Ingestion)

In [3]:
# https://www.youtube.com/watch?v=o126p1QN_RI&t=6198s

In [10]:
help(YouTubeTranscriptApi)

Help on class YouTubeTranscriptApi in module youtube_transcript_api._api:

class YouTubeTranscriptApi(builtins.object)
 |  YouTubeTranscriptApi(proxy_config: Optional[youtube_transcript_api.proxies.ProxyConfig] = None, http_client: Optional[requests.sessions.Session] = None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, proxy_config: Optional[youtube_transcript_api.proxies.ProxyConfig] = None, http_client: Optional[requests.sessions.Session] = None)
 |      Note on thread-safety: As this class will initialize a `requests.Session`
 |      object, it is not thread-safe. Make sure to initialize an instance of
 |      `YouTubeTranscriptApi` per thread, if used in a multi-threading scenario!
 |      
 |      :param proxy_config: an optional ProxyConfig object, defining proxies used for
 |          all network requests. This can be used to work around your IP being blocked
 |          by YouTube, as described in the "Working around IP bans" section of the
 |          README
 |      

In [27]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "o126p1QN_RI"

try:
    # Create an API instance and fetch the transcript
    ytt_api = YouTubeTranscriptApi()
    fetched_transcript = ytt_api.fetch(video_id, languages=['en'])
    
    # Correctly extract text from the FetchedTranscriptSnippet objects
    transcript = " ".join(chunk.text for chunk in fetched_transcript)
    print(transcript)

except Exception as e:
    print(f"An error occurred: {e}")

Hello all, my name is Krishna and I am super excited to announce this amazing crash course on rag that is retrieval augmented generation. Uh in this specific crash course it'll be somewhere around 2.5 to 3 hours but we are going to discuss everything that is related to rack completely from scratch. Uh we'll be talking about the entire pipeline from data injection to retrieval pipeline to output generation. how to use LLM models, how to use embedding models in this uh along with this uh what should be the right strategy of using chunkings and many more things right so we will be deep diving into both the theoretical understanding along with the practical implementation and we will initially go ahead step by step we'll start with the basic implementation and then as we go ahead in the advanced section we'll also implement the modular coding right the main aim of the modular coding is to link the entire pipeline in a way so that you should be able to understand how rag actually works and 

In [29]:
for chunk in fetched_transcript: 
    print(chunk)

FetchedTranscriptSnippet(text='Hello all, my name is Krishna and I am', start=0.56, duration=4.56)
FetchedTranscriptSnippet(text='super excited to announce this amazing', start=3.04, duration=4.719)
FetchedTranscriptSnippet(text='crash course on rag that is retrieval', start=5.12, duration=5.28)
FetchedTranscriptSnippet(text='augmented generation. Uh in this', start=7.759, duration=4.241)
FetchedTranscriptSnippet(text="specific crash course it'll be somewhere", start=10.4, duration=4.399)
FetchedTranscriptSnippet(text='around 2.5 to 3 hours but we are going', start=12.0, duration=5.359)
FetchedTranscriptSnippet(text='to discuss everything that is related to', start=14.799, duration=5.281)
FetchedTranscriptSnippet(text="rack completely from scratch. Uh we'll", start=17.359, duration=4.801)
FetchedTranscriptSnippet(text='be talking about the entire pipeline', start=20.08, duration=4.24)
FetchedTranscriptSnippet(text='from data injection to retrieval', start=22.16, duration=4.72)
FetchedT

## Step 1b - Indexing (Text Splitting)

In [30]:
splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)

In [31]:
chunks=splitter.split_text(transcript)

In [33]:
chunks[0]

"Hello all, my name is Krishna and I am super excited to announce this amazing crash course on rag that is retrieval augmented generation. Uh in this specific crash course it'll be somewhere around 2.5 to 3 hours but we are going to discuss everything that is related to rack completely from scratch. Uh we'll be talking about the entire pipeline from data injection to retrieval pipeline to output generation. how to use LLM models, how to use embedding models in this uh along with this uh what should be the right strategy of using chunkings and many more things right so we will be deep diving into both the theoretical understanding along with the practical implementation and we will initially go ahead step by step we'll start with the basic implementation and then as we go ahead in the advanced section we'll also implement the modular coding right the main aim of the modular coding is to link the entire pipeline in a way so that you should be able to understand how rag actually works and

In [34]:
type(chunks[0])

str

In [35]:
type(chunks)

list

In [38]:
chunks=splitter.create_documents([transcript])

In [39]:
chunks

[Document(metadata={}, page_content="Hello all, my name is Krishna and I am super excited to announce this amazing crash course on rag that is retrieval augmented generation. Uh in this specific crash course it'll be somewhere around 2.5 to 3 hours but we are going to discuss everything that is related to rack completely from scratch. Uh we'll be talking about the entire pipeline from data injection to retrieval pipeline to output generation. how to use LLM models, how to use embedding models in this uh along with this uh what should be the right strategy of using chunkings and many more things right so we will be deep diving into both the theoretical understanding along with the practical implementation and we will initially go ahead step by step we'll start with the basic implementation and then as we go ahead in the advanced section we'll also implement the modular coding right the main aim of the modular coding is to link the entire pipeline in a way so that you should be able to u

In [40]:
chunks[0]

Document(metadata={}, page_content="Hello all, my name is Krishna and I am super excited to announce this amazing crash course on rag that is retrieval augmented generation. Uh in this specific crash course it'll be somewhere around 2.5 to 3 hours but we are going to discuss everything that is related to rack completely from scratch. Uh we'll be talking about the entire pipeline from data injection to retrieval pipeline to output generation. how to use LLM models, how to use embedding models in this uh along with this uh what should be the right strategy of using chunkings and many more things right so we will be deep diving into both the theoretical understanding along with the practical implementation and we will initially go ahead step by step we'll start with the basic implementation and then as we go ahead in the advanced section we'll also implement the modular coding right the main aim of the modular coding is to link the entire pipeline in a way so that you should be able to un

In [41]:
type(chunks[0])

langchain_core.documents.base.Document

In [42]:
type(chunks)

list

In [47]:
len(chunks)

151

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [44]:
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store=FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

In [46]:
vector_store.index_to_docstore_id

{0: '4c45099e-bc08-4967-a5a8-315943fc847d',
 1: '156f6298-56ea-4550-a684-091a549adc25',
 2: '9adac588-5765-4254-9042-d231be476f05',
 3: '5f075923-af12-4efb-85e3-0b2a92007faf',
 4: 'df27c3e2-2623-4e20-b541-d8c1d18befa3',
 5: 'e8202b12-8aaa-474d-a5e3-fc5002ea2cfe',
 6: 'a72c8550-e673-4033-b935-04357e39c0ae',
 7: '5f86c7f5-6468-48d4-b7ce-b7934c36ed6b',
 8: '3a04127f-d1a1-45ed-87f2-32611f66f6c4',
 9: 'a17c2f23-cd21-44cd-90fd-27a4f9b84e48',
 10: '9720fb8d-64d5-477e-919d-31d25e13ac65',
 11: '80e98d5e-e793-45e0-9075-fa3e58bfe87d',
 12: '41a414ec-4e1d-47c6-918b-c9b28f55b1e9',
 13: '9a8e9129-4492-419d-b701-939482b583c2',
 14: 'b7237824-794d-4909-a671-26bfa95095cd',
 15: '9af7a1ad-a5de-4abd-9597-4e35117b6ae5',
 16: '3896bb9d-826e-4baf-a7a0-b37165027771',
 17: 'cb274b7f-9bd1-4789-bfad-7b27fff2ea38',
 18: '52f101e3-13f7-4a9f-9738-0ae2ae232a35',
 19: '889c684a-ba16-4a2c-aee7-1609dbce4902',
 20: '89fbacc9-147d-40e9-b4c0-66e4e1712093',
 21: '7c41cdfd-e7ce-4ace-8854-446df598d932',
 22: '4d341b0b-35bd-

In [49]:
vector_store.get_by_ids(['4c45099e-bc08-4967-a5a8-315943fc847d'])

[Document(id='4c45099e-bc08-4967-a5a8-315943fc847d', metadata={}, page_content="Hello all, my name is Krishna and I am super excited to announce this amazing crash course on rag that is retrieval augmented generation. Uh in this specific crash course it'll be somewhere around 2.5 to 3 hours but we are going to discuss everything that is related to rack completely from scratch. Uh we'll be talking about the entire pipeline from data injection to retrieval pipeline to output generation. how to use LLM models, how to use embedding models in this uh along with this uh what should be the right strategy of using chunkings and many more things right so we will be deep diving into both the theoretical understanding along with the practical implementation and we will initially go ahead step by step we'll start with the basic implementation and then as we go ahead in the advanced section we'll also implement the modular coding right the main aim of the modular coding is to link the entire pipeli

## Step 2 - Retrieval

In [50]:
retriever=vector_store.as_retriever(
    search_type='similarity', 
    search_kwargs={'k': 4}
)

In [51]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029EC89B9930>, search_kwargs={'k': 4})

In [52]:
type(retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [53]:
retriever.invoke('what is rag?')

[Document(id='cab2dc77-5b97-4f7f-a400-bc8e09ad220c', metadata={}, page_content="this was a complete idea or a kind of crash course that I really wanted to give on the entire uh rag. Rag is one of the most important use cases. That is what I always believe. Most of the companies are specifically building rag applications. So I think this is really really important and super cool topic. I hope you like this particular video. This was it from my side. I'll see you on the next video. Thank you. Take care."),
 Document(id='9adac588-5765-4254-9042-d231be476f05', metadata={}, page_content="of all we'll try to understand rag. Okay. So first of all let's go through the definition and then I will give you a brief idea what exactly rag is all about you know. So here you can clearly see that rag is the process of optimizing the output of a large language model. Okay. So it references an authorative knowledge base outside of his training data set source before get generating a response. LLMs are tr

## Step 3 - Augmentation

In [101]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.2)

In [55]:
prompt=PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables=['context', 'question']
)

In [57]:
question = "is the topic of retriever discussed in this video? if yes then what was discussed"
retrieved_docs=retriever.invoke(question)

In [82]:
retrieved_docs

[Document(id='b8230466-6a19-4a33-9798-e6dea8aaf435', metadata={}, page_content="see that it handles query based retrieval from the vector store. So inside the constructor we will be giving two important parameters. One is the vector store and one is the embedding manager. And if you remember we have created both this. We have created the embedding manager. We have created the vector store manager. Right now after giving this we will be initializing two class variables that is vector store and embedding manager and we'll be assigning with this. Now whenever we create a retriever one thing you really need to understand this retriever is actually built on the top of a vector store and retriever is nothing but it is a simple interface based on whatever query we get this retriever is just going to give you the response back. Okay and this retriever is basically a kind of interface which is connected to the vector store and chart. Okay. Now uh the next step that we are going to create is ano

In [83]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [84]:
context_text

"see that it handles query based retrieval from the vector store. So inside the constructor we will be giving two important parameters. One is the vector store and one is the embedding manager. And if you remember we have created both this. We have created the embedding manager. We have created the vector store manager. Right now after giving this we will be initializing two class variables that is vector store and embedding manager and we'll be assigning with this. Now whenever we create a retriever one thing you really need to understand this retriever is actually built on the top of a vector store and retriever is nothing but it is a simple interface based on whatever query we get this retriever is just going to give you the response back. Okay and this retriever is basically a kind of interface which is connected to the vector store and chart. Okay. Now uh the next step that we are going to create is another function which will be called as retrieve function. Now this is really\n\n

In [85]:
final_prompt=prompt.invoke({'context':context_text, 'question':question})

In [86]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      see that it handles query based retrieval from the vector store. So inside the constructor we will be giving two important parameters. One is the vector store and one is the embedding manager. And if you remember we have created both this. We have created the embedding manager. We have created the vector store manager. Right now after giving this we will be initializing two class variables that is vector store and embedding manager and we'll be assigning with this. Now whenever we create a retriever one thing you really need to understand this retriever is actually built on the top of a vector store and retriever is nothing but it is a simple interface based on whatever query we get this retriever is just going to give you the response back. Okay and this retriever is basically a kind of interface 

## Step 4 - Generation

In [87]:
answer=llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of a retriever is discussed.

Based on the transcript, here is what was discussed:

*   A retriever is described as a simple interface built on top of a vector store.
*   It handles query-based retrieval from the vector store.
*   The process involves taking a user query, converting it into embeddings, and then using the retriever to hit the vector store and get the context back.
*   The video discusses creating a specific `rag retriever` class. The constructor for this class takes two parameters: the vector store and the embedding manager.
*   This class will also have a `retrieve` function.


## Building a Chain

In [88]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [89]:
def format_docs(retrieved_docs): 
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [90]:
parallel_chain=RunnableParallel({
    'context': retriever | RunnableLambda(format_docs), 
    'question': RunnablePassthrough()
})

In [92]:
parallel_chain.invoke('what is rag')

{'context': "this was a complete idea or a kind of crash course that I really wanted to give on the entire uh rag. Rag is one of the most important use cases. That is what I always believe. Most of the companies are specifically building rag applications. So I think this is really really important and super cool topic. I hope you like this particular video. This was it from my side. I'll see you on the next video. Thank you. Take care.\n\nof all we'll try to understand rag. Okay. So first of all let's go through the definition and then I will give you a brief idea what exactly rag is all about you know. So here you can clearly see that rag is the process of optimizing the output of a large language model. Okay. So it references an authorative knowledge base outside of his training data set source before get generating a response. LLMs are trained on vast volume of data as we all know and use billions of parameters to generally original output for task like question answering, translati

In [93]:
parser=StrOutputParser()

In [94]:
main_chain=parallel_chain | prompt | llm | parser

In [103]:
result=main_chain.invoke('can you summarize the video in 200 words')
print(result)

Based on the transcript provided, the video discusses three types of RAG (Retrieval-Augmented Generation) pipelines: a simple pipeline, an enhanced one, and a final version that incorporates streaming, citation, history, and summarization.

An example of an "rag advanced function" is shown, where a query like "what is the attention mechanism?" is executed. The system returns the answer along with source information, including page numbers, a similarity score, and a preview of the context.

The speaker mentions that the application Perplexity is a RAG-based application that connects to various retrievers, tools, and web search. The speaker also reveals they are developing their own RAG application for a startup.

Future videos will cover more advanced topics such as modular coding, optimization, various chunking strategies like semantic chunking, and context engineering. The overall goal of the video is to provide a clear understanding of what a RAG pipeline is and how it works.


In [105]:
retrieved_docs

[Document(id='b8230466-6a19-4a33-9798-e6dea8aaf435', metadata={}, page_content="see that it handles query based retrieval from the vector store. So inside the constructor we will be giving two important parameters. One is the vector store and one is the embedding manager. And if you remember we have created both this. We have created the embedding manager. We have created the vector store manager. Right now after giving this we will be initializing two class variables that is vector store and embedding manager and we'll be assigning with this. Now whenever we create a retriever one thing you really need to understand this retriever is actually built on the top of a vector store and retriever is nothing but it is a simple interface based on whatever query we get this retriever is just going to give you the response back. Okay and this retriever is basically a kind of interface which is connected to the vector store and chart. Okay. Now uh the next step that we are going to create is ano

In [ ]:
# 💡 In short:

# k=4 → retrieves top 4 relevant chunks.

# It does not summarize the whole video, only those parts.
# For a complete summary, you need:
        # chunk-wise summarization, or
        # map-reduce chain, or
        # increase k.